In [4]:
## Make width of notebook 100%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
from datasets import data as Local ## Local module to get dataframes

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [11]:
games_df, winners_df, cumulative_stats_df, games_details_df, teams_df, players_df, ranking_df = Local.get_all_dataset_dataframes('../../datasets/')

In [12]:
winners_df # Whether the home team won in a game in each row of `games_df`

16157    1
16156    1
16155    1
16154    1
16153    1
        ..
4        0
3        0
2        0
1        1
0        1
Name: HOME_TEAM_WINS, Length: 23520, dtype: int64

In [13]:
games_df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away
16157,2003-10-05,10300001,1610612762,1610612742,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0
16156,2003-10-06,10300002,1610612763,1610612749,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0
16155,2003-10-07,10300009,1610612758,1610612746,2003,101.0,0.467,0.871,0.444,19.0,39.0,82.0,0.368,0.609,0.364,13.0,50.0
16154,2003-10-07,10300005,1610612757,1610612745,2003,104.0,0.527,0.657,0.429,22.0,33.0,80.0,0.470,0.667,0.333,10.0,37.0
16153,2003-10-07,10300007,1610612748,1610612755,2003,86.0,0.352,0.647,0.222,15.0,55.0,79.0,0.329,0.897,0.143,7.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2020-12-18,12000040,1610612761,1610612748,2020,105.0,0.380,0.737,0.356,27.0,37.0,117.0,0.534,0.741,0.514,30.0,51.0
3,2020-12-18,12000039,1610612754,1610612755,2020,107.0,0.371,0.692,0.262,19.0,45.0,113.0,0.533,0.629,0.355,23.0,48.0
2,2020-12-19,12000049,1610612763,1610612737,2020,116.0,0.400,0.744,0.396,21.0,43.0,117.0,0.422,0.837,0.297,24.0,47.0
1,2020-12-19,12000048,1610612764,1610612765,2020,99.0,0.427,0.625,0.295,24.0,45.0,96.0,0.402,0.647,0.326,18.0,51.0


In [35]:
cumulative_stats_df

,DATE,GAME_ID,TEAM_ID,SEASON_TYPE,SEASON,AT_HOME,PTS_total,AST_total,GAMES_PLAYED,REB_total,RPG_total,FG3_PCT_total,FG_PCT_total,APG_total,FT_PCT_total,PPG_total
0,2003-10-05,10300001,1610612762,1,2003,True,90.0,23.0,1,41.0,41.000000,0.143000,0.45700,23.000000,0.73500,90.000000
1,2003-10-05,10300001,1610612742,1,2003,False,85.0,20.0,1,38.0,38.000000,0.250000,0.44700,20.000000,0.50000,85.000000
2,2003-10-06,10300002,1610612763,1,2003,True,105.0,25.0,1,48.0,48.000000,0.267000,0.49400,25.000000,0.61800,105.000000
3,2003-10-06,10300002,1610612749,1,2003,False,94.0,20.0,1,43.0,43.000000,0.154000,0.42700,20.000000,0.70000,94.000000
4,2003-10-07,10300009,1610612758,1,2003,True,101.0,19.0,1,39.0,39.000000,0.444000,0.46700,19.000000,0.87100,101.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47035,2020-12-19,12000049,1610612737,1,2020,True,451.0,96.0,4,215.0,53.750000,0.323500,0.40000,24.000000,0.80150,112.750000
47036,2020-12-19,12000048,1610612764,1,2020,False,299.0,68.0,3,134.0,44.666667,0.312667,0.41900,22.666667,0.67800,99.666667
47037,2020-12-19,12000048,1610612765,1,2020,True,376.0,78.0,4,176.0,44.000000,0.331250,0.41050,19.500000,0.69650,94.000000
47038,2020-12-19,12000047,1610612753,1,2020,False,458.0,98.0,4,204.0,51.000000,0.335250,0.44150,24.500000,0.76650,114.500000


## Remove non-data

In [37]:
cumulative_stats_copy_df = cumulative_stats_df.copy()
for key in ['DATE','GAME_ID','TEAM_ID','SEASON_TYPE','SEASON']:
    del cumulative_stats_copy_df[key]
# Only look at home teams
# cumulative_stats_copy_df = cumulative_stats_copy_df.loc[cumulative_stats_copy_df['AT_HOME']==True]
cumulative_stats_copy_df = cumulative_stats_copy_df.loc[cumulative_stats_copy_df['AT_HOME'].index%2==0]
del cumulative_stats_copy_df['AT_HOME']
cumulative_stats_copy_df

,PTS_total,AST_total,GAMES_PLAYED,REB_total,RPG_total,FG3_PCT_total,FG_PCT_total,APG_total,FT_PCT_total,PPG_total
0,90.0,23.0,1,41.0,41.000000,0.143000,0.457000,23.000000,0.735000,90.000000
2,105.0,25.0,1,48.0,48.000000,0.267000,0.494000,25.000000,0.618000,105.000000
4,101.0,19.0,1,39.0,39.000000,0.444000,0.467000,19.000000,0.871000,101.000000
6,104.0,22.0,1,33.0,33.000000,0.429000,0.527000,22.000000,0.657000,104.000000
8,86.0,15.0,1,55.0,55.000000,0.222000,0.352000,15.000000,0.647000,86.000000
...,...,...,...,...,...,...,...,...,...,...
47030,328.0,77.0,3,120.0,40.000000,0.354000,0.422333,25.666667,0.706333,109.333333
47032,317.0,71.0,3,116.0,38.666667,0.342667,0.415667,23.666667,0.643333,105.666667
47034,474.0,113.0,4,201.0,50.250000,0.346000,0.450000,28.250000,0.714500,118.500000
47036,299.0,68.0,3,134.0,44.666667,0.312667,0.419000,22.666667,0.678000,99.666667


In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    cumulative_stats_copy_df,
    winners_df,
    random_state=0,
)

In [41]:
# Get best params based from CV function
def do_cv(classifier, cv_fun, params, x, y, **kwargs):
    clf = cv_fun(
        classifier,
        params,
        scoring='accuracy', # Optimize for the 'accuracy' score of the model.
        **kwargs,
    )
    clf.fit(x, y)
    return clf
def print_best_decision_tree(params, X_train, y_train, X_test, y_test):
    cv_clf = do_grid_cv(
        DecisionTreeClassifier(random_state=0),
        RandomizedSearchCV(random_state=0),
        params,
        X_train,
        y_train,
    )
    print('Best decision tree accuracy (%s): %.2f' % (str(cv_clf.best_params_), cv_clf.score(X_test, y_test)))
    return cv_clf

In [42]:
decision_tree_clf = DecisionTreeClassifier(random_state=0)
decision_tree_clf.fit(X_train, y_train)
print('Decision tree accuracy (no pruning): %.2f' % decision_tree_clf.score(X_test, y_test))

Decision tree accuracy (no pruning): 0.56
